In [1]:
from __future__ import annotations

# Download and install English language model for spaCy

In [2]:
! python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------------------- -------------------- 6.3/12.8 MB 31.1 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 32.6 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 32.6 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 32.6 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 14.3 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Import all libraries

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Import data

In [4]:
df = pd.read_csv('starter/data/reviews.csv')
df.head()

,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name,Recommended IND
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses,0
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants,1
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses,1
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses,0
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits,1


# DF stats

In [5]:
df.describe()

,Clothing ID,Age,Positive Feedback Count,Recommended IND
count,18442.000000,18442.000000,18442.000000,18442.000000
mean,954.896757,43.383635,2.697484,0.816235
std,141.571783,12.246264,5.942220,0.387303
min,2.000000,18.000000,0.000000,0.000000
25%,863.000000,34.000000,0.000000,1.000000
50%,952.000000,41.000000,1.000000,1.000000
75%,1078.000000,52.000000,3.000000,1.000000
max,1205.000000,99.000000,122.000000,1.000000


# Check Nulls

In [6]:
df.isnull().sum()

Clothing ID                0
Age                        0
Title                      0
Review Text                0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
Recommended IND            0
dtype: int64

# Separate features from labels

In [7]:
X = df.drop('Recommended IND' , axis= 1)
Y = df['Recommended IND'].copy()

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.2, random_state=27, shuffle=True)

# Split data into numerical, categorical, and text features

In [8]:
# Select columns that are numeric 
num_features = X.select_dtypes(exclude=['object']).columns
print('Numerical features:', num_features)

# Select columns that are categorical 
cat_features = X.select_dtypes(include=['object']).columns
print('Categorical features:', cat_features)

# Select column with review text
text_features = X[['Review Text']].columns
print('Review Text features:', text_features)

# Show first rows of the dataset to check
df.head()

Numerical features: Index(['Clothing ID', 'Age', 'Positive Feedback Count'], dtype='object')
Categorical features: Index(['Title', 'Review Text', 'Division Name', 'Department Name',
       'Class Name'],
      dtype='object')
Review Text features: Index(['Review Text'], dtype='object')


,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name,Recommended IND
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses,0
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants,1
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses,1
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses,0
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits,1


# Pipeline for num features

In [9]:
num_pipeline = Pipeline([
    # fill missing values with most frequent value
    ('imputer', SimpleImputer(strategy='median')),  
    
    # scale numbers to range 0-1
    ('scaler', MinMaxScaler())
])

num_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', MinMaxScaler())])

# Pipeline for cat features

In [10]:
# Pipeline for cat features
cat_pipeline = Pipeline([     
    # fill missing values with most frequent value
    ('imputer', SimpleImputer(strategy='most_frequent')),
    
    # create one-hot columns (0/1) for each category
    ('cat_encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
])

cat_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('cat_encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

# Text features, Count Characters Class

In [11]:
class CountCharacter(BaseEstimator, TransformerMixin):
    def __init__(self, character: str):
        # Character we want to count
        self.character = character

    def fit(self, X, y=None):
        # Just return self
        return self

    def transform(self, X):
        # Count how many times the character appears in each text
        # (text or "") -> handle None values as empty string
        return [[(text or "").count(self.character)] for text in X]

# Preprocess text to make it 1D for transformers

In [12]:
initial_text_preprocess = Pipeline([('dimension_reshaper', FunctionTransformer(lambda x: x.values.ravel()))])

# Create features by counting specific characters
feature_engineering = FeatureUnion([
    ('count_spaces', CountCharacter(character=' ')),      # count spaces
    ('count_exclamations', CountCharacter(character='!')), # count exclamation marks
    ('count_question_marks', CountCharacter(character='?')), # count question marks
])

# Combine preprocessing and feature engineering into one pipeline
character_counts_pipeline = Pipeline([
    ('initial_text_preprocess', initial_text_preprocess), # reshape text
    ('feature_engineering', feature_engineering),         # count characters
])

character_counts_pipeline

Pipeline(steps=[('initial_text_preprocess',
                 Pipeline(steps=[('dimension_reshaper',
                                  FunctionTransformer(func=<function <lambda> at 0x00000161B869C400>))])),
                ('feature_engineering',
                 FeatureUnion(transformer_list=[('count_spaces',
                                                 CountCharacter(character=' ')),
                                                ('count_exclamations',
                                                 CountCharacter(character='!')),
                                                ('count_question_marks',
                                                 CountCharacter(character='?'))]))])

# custom transformer for lemmatizing text and removing stopwords

In [13]:
class SpacyLemmatizer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        # Store the spaCy model (nlp) to use for processing text
        self.nlp = nlp

    def fit(self, X, y=None):
        # just return self
        return self

    def transform(self, X):
        # Process each text in X
        lemmatized = [
            ' '.join(
                token.lemma_ for token in doc  # take the base form of each word
                if not token.is_stop           # skip stopwords like "the", "and", "is"
            )
            for doc in self.nlp.pipe(X, batch_size=50)        # use spaCy to process all texts batch size - performance
        ]
        return lemmatized                     # return list of cleaned, lemmatized texts

# TF-IDF features

In [14]:
tfidf_pipeline = Pipeline([
    # Reshape input data to a 1D array
    ('dimension_reshaper', FunctionTransformer(np.reshape, kw_args={'newshape': -1})),
    
    # Lemmatize the text using spaCy
    ('lemmatizer', SpacyLemmatizer(nlp=nlp)),
    
    # Convert processed text into a TF-IDF matrix
    ('tfidf_vectorizer', TfidfVectorizer())  
])
tfidf_pipeline

Pipeline(steps=[('dimension_reshaper',
                 FunctionTransformer(func=<function reshape at 0x00000161A971DA70>,
                                     kw_args={'newshape': -1})),
                ('lemmatizer',
                 SpacyLemmatizer(nlp=<spacy.lang.en.English object at 0x00000161AABFFB60>)),
                ('tfidf_vectorizer', TfidfVectorizer())])

# Combine all feature processing steps into one transformer

In [15]:
full_preprocessor = ColumnTransformer([
    # Numeric pipeline: applies transformations to numerical features
    ('num', num_pipeline, num_features),
    
    # Categorical pipeline: handles encoding/processing of categorical features
    ('cat', cat_pipeline, cat_features),
    
    # Character counts pipeline: extracts custom features such as text length,
    ('character_counts', character_counts_pipeline, text_features),
    
    # TF-IDF text pipeline: transforms text columns into TF-IDF vectors
    ('tfidf_text', tfidf_pipeline, text_features),
])
feature_engineering

FeatureUnion(transformer_list=[('count_spaces', CountCharacter(character=' ')),
                               ('count_exclamations',
                                CountCharacter(character='!')),
                               ('count_question_marks',
                                CountCharacter(character='?'))])

# Train Model

In [16]:
# Create a complete machine learning pipeline
model_pipeline = make_pipeline(
    
    # Preprocessing - applies all feature engineering transformations
    full_preprocessor,
    
     # Model - a Random Forest classifier
    RandomForestClassifier(random_state=27)
)

# Fit the pipeline on the training data
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['Clothing ID', 'Age', 'Positive Feedback Count'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('cat_encoder',
                                                                   OneHotEncoder(handle_unknow...
                                                  Pipeline(steps=[('dimension_reshaper',
                                                                   FunctionTransformer(func=<function reshape at 0x00000161A971DA70>,
                                                                                       kw_args={'newshape': -1})),
                                                                  ('lemmatizer',
                                                                   SpacyLemmatizer(nlp=<spacy.lang.en.English object at 0x00000161AABFFB60>)),
                                                                  ('tfidf_vectorizer',
                                                                   TfidfVectorizer())]),
                                                  Index(['Review Text'], dtype='object'))])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=27))])

# Evaluate

In [17]:
# Make predictions on the test set using the trained pipeline
y_pred = model_pipeline.predict(X_test)

# Print accuracy score (overall percentage of correct predictions)
print('Accuracy:', accuracy_score(y_test, y_pred))

# Print classification report (precision, recall, f1-score, support per class)
print('\nClassification Report:\n', classification_report(y_test, y_pred))

# Print confusion matrix (shows correct vs. incorrect predictions for each class)
print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred))

c:\Users\R44063\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Accuracy: 0.8460287340742749

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.13      0.23       638
           1       0.85      1.00      0.91      3051

    accuracy                           0.85      3689
   macro avg       0.86      0.56      0.57      3689
weighted avg       0.85      0.85      0.80      3689


Confusion Matrix:
 [[  83  555]
 [  13 3038]]


# Fine-Tune Model

In [18]:
# Define the hyperparameter search space for RandomForestClassifier
param_distributions = dict(
    randomforestclassifier__max_features=[100, 150, 250],
    randomforestclassifier__n_estimators=[150, 200],
)

# Create a randomized search with cross-validation
param_search = RandomizedSearchCV(
    estimator=model_pipeline,
    param_distributions=param_distributions,
    n_iter=6,
    cv=5,
    n_jobs=-1,
    refit=True,
    verbose=3,
    random_state=27
)

# Perform the randomized search on training data
param_search.fit(X_train, y_train)


print('Best Params:', param_search.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [ ]:
# Extract the best pipeline (with the best hyperparameters) from the search
model_best = param_search.best_estimator_

# Use the tuned model to make predictions on the test set
y_pred_best = model_best.predict(X_test)

In [ ]:
# Print accuracy score of the tuned model
print('Tuned Accuracy:', accuracy_score(y_test, y_pred_best))

# Print classification report (precision, recall, f1-score for each class)
print('\nTuned Classification Report:\n', classification_report(y_test, y_pred_best))

# Print confusion matrix (comparison of true vs. predicted labels)
print('\nTuned Confusion Matrix:\n', confusion_matrix(y_test, y_pred_best))


c:\Users\R44063\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Accuracy: 0.8517213336947682
